In [476]:
with open("trainingData.txt") as f:
    mydata=[[value for value in line.split()]for line in f]
with open("validationData.txt") as f:
    mydata2=[[value for value in line.split()]for line in f]

In [477]:
import numpy as np

def modify(xtrain):
    chemolist={}
    for i in range(len(xtrain)):
        for j in range(len(xtrain[i])):

            if xtrain[i][j] == 'YES' or xtrain[i][j]=='Yes':
                xtrain[i][j]=1
            elif xtrain[i][j]=='NO' or xtrain[i][j]=='No':
                xtrain[i][j]=-1
            elif xtrain[i][j] =='POS':
                xtrain[i][j]=1
            elif xtrain[i][j]=='NEG':
                xtrain[i][j]=-1
            elif xtrain[i][j]=='ND' or xtrain[i][j]=='NotDone':
                xtrain[i][j]=0
            elif xtrain[i][j]=='F':
                xtrain[i][j]=1
            elif xtrain[i][j]=='M':
                xtrain[i][j]=2
        
        if xtrain[i][11] not in chemolist:
            chemolist[xtrain[i][11]]=1
        else:
            chemolist[xtrain[i][11]]+=1
            
    for i in range(len(xtrain)):
        xtrain[i][11]=chemolist.get(xtrain[i][11])/166
    return xtrain

def Ymodify(ytrain):
    y=[]
    for i in range(len(ytrain)):
        if ytrain[i][0]=='COMPLETE_REMISSION':
            y.append(0)
        else:
            y.append(1)
    return y

In [478]:
label=mydata[1]
data=mydata[2:]
# data=[]
# for i in range(len(d)):
#     if d[i][266]=="COMPLETE_REMISSION":
#         data.append(d[i])

xtrain=[]
ytrain=[]
for i in range(len(data)):
    xtrain.append(data[i][:266])
    ytrain.append(data[i][266:])
x=modify(xtrain)
Y=Ymodify(ytrain)
#Y=ytrain

patient=[]
for i in range(len(x)):
    patient_data = x[i][1:]
    for j in range(len(patient_data)):
        if patient_data[j]=='NA':
            patient_data[j]=0.0
        else:
            patient_data[j]=float(patient_data[j])
    patient.append(patient_data)

X=np.array(patient)
Y=np.array(Y)

In [268]:
print(np.random.seed(1))

None


In [367]:
def activation(x):
    result=(1.0 / (1.0 + np.exp(-x)))
    #print('result')
    #print(result.shape)
    return result
def derivative(x):
    x*(1-x)
    return x
def calculate_cost(Y,layerOutput,weights1,weights2):
    A = -(layerOutput - Y)
    B = -(layerOutput - 1)
    o = layerOutput
    error=layerOutput
#     print("A"+str(A.shape))
#     print("B"+str(B.shape))
#     print("O"+str(layerOutput.shape))
#     print("E"+str(error.shape))
    for i in range(len(A)):
        error[i]=o[i]*B[i]*A[i]
        #print(error[i])
#     #error = (1.0/2.0)*((Y-layerOutput)**2)
#     error=layerOutput
#     for i in range(len(layerOutput)):
#         error[i]  = (1.0/2.0)*((Y-layerOutput[i])**2)
    return error
def forward(x,w1,w2):
    InputLayer=(np.array([x]))
    #print('InputLayer')
    #print(InputLayer.shape)
    output1=np.dot(InputLayer,w1.T)
    layer1=activation(output1)
    output2=np.dot(layer1,w2.T)
    layer2=activation(output2)
    return layer1,layer2
    

def neural_net(xtrain,ytrain,learningRate):
    # Standardize X:
    #X = xtrain
    #X[:, 1] = (xtrain[:, 1]-np.mean(xtrain[:, 1]))/np.std(xtrain[:, 1])
    #print(X)
    #print('xtrain')
    #print(xtrain)
    X = xtrain
    print(X.shape)
    Y = (np.array([ytrain])).T
    #Y=ytrain
    #Z = (np.array([Y])).T
    #print('Y')
    #print(Y.shape)
    #print((np.asarray(Y)))
    #print(Z.shape)
    
   
    
    np.random.seed(1)
    hidden=60
    w1_ = np.random.uniform(-0.5, 0.5, size=hidden*(X.shape[1]))
    syn0 = w1_.reshape(hidden, X.shape[1])
    w2_ = np.random.uniform(-0.5, 0.5, size=2*(hidden))
    syn1 = w2_.reshape(2,hidden)
    
    for j in range(1000):
        save0=syn0
        save1=syn1
        
        for i in range(len(X)):
            inputX=np.array(X[i])
            o1,o2=forward(inputX,syn0,syn1)
            cost=calculate_cost(Y[i],o2,syn0,syn1)
#             if (j%10000)==0:
#                 print("Error:"+ str(np.mean(np.abs(cost))))
            
            #print(cost.shape) # (1,1)
#             print(o1.shape) # (1,50)
            #print(o2.shape) # (1,1)
            #print(syn0.shape) # (50,265)
            #print(syn1.shape) # (1,50)
#             z=activation(o1)
#             sg=derivative(z)
            error_l2 = np.dot(l2,(1-l2).T) #(1,1)
            value = np.dot(syn1.T,cost.T) #(50,1)
            
            #delta_l2 = np.dot(error_l2.T,cost) # (50,1)
#             delta_l1 = sg * np.dot(cost,syn1)
            delta_l1 = np.dot(value,error_l2)
    
            syn0 += learningRate * (np.dot(delta_l1,((np.array([inputX])))))
            syn1 += learningRate * (np.dot(cost.T,l1))
            
#         var0=np.mean(np.abs(save0-syn0))
#         var1=np.mean(np.abs(save1-syn1))
        
#         if (var0) < precision:
#             return syn0, syn1
        
    return syn0, syn1

In [368]:
def predit(w0,w1,test):
    InputLayer=(np.array([test]))
    layer1=activation(np.dot(InputLayer,w0.T))
    layer2=activation(np.dot(layer1,w1.T))
    #InputLayer=test
    #layer1=activation(np.dot(InputLayer,w0))
    #layer2=activation(np.dot(layer1,w1))
    return layer2

In [370]:
Xtrain=X[:150]
Ytrain=Y[:150]
Xvalid=X[150:]
Yvalid=Y[150:]
(w0,w1)=neural_net(Xtrain,Ytrain,0.0005)
for i in range(len(Xvalid)):
    y=Yvalid[i]
    Xvalidarray = (np.array([Xvalid[i]]))
    x=predit(w0,w1,Xvalidarray)
    print(str(np.argmax(x)) + ', ' + str(y))

(150, 265)
0, 0
0, 0
0, 1
0, 1
0, 0
0, 1
0, 0
0, 1
0, 0
0, 1
0, 1
0, 1
0, 1
0, 0
0, 1
0, 0


In [376]:
import numpy as np
np.set_printoptions(threshold=numpy.nan)

NameError: name 'numpy' is not defined

In [377]:
with open("w0.txt", 'w') as file:
    for item in w0:
        file.write("{}\n".format(item))
        

In [348]:
def predit(w0,w1,test):
    InputLayer=(np.array([test]))
    layer1=activation(np.dot(InputLayer,w0.T))
    layer2=activation(np.dot(layer1,w1.T))
    #InputLayer=test
    #layer1=activation(np.dot(InputLayer,w0))
    #layer2=activation(np.dot(layer1,w1))
    return layer2
    
test=modify(mydata2)

patient=[]
for i in range(len(test)):
    patient_data = test[i][1:]
    for j in range(len(patient_data)):
        if patient_data[j]=='NA':
            patient_data[j]=0.0
        else:
            patient_data[j]=float(patient_data[j])
    patient.append(patient_data)

test=np.array(patient)
print(len(test[1]))
for i in range(len(test)):
    result=predit(w0,w1,test[i])
    print(result)


265
[[ 0.99393591]]
[[ 0.9928511]]
[[ 0.99360772]]
[[ 0.99389775]]
[[ 0.9939785]]


#Linear regression

In [423]:
X=np.array(patient,dtype=float)
Y=np.array(ytrain)
y1=Y[:,2]
y1=np.array(y1,dtype=float)
print(y1)

[ 568.57  185.86   56.29   98.14  454.71   50.    734.86  690.14  410.14
  703.86   75.43   26.71  205.43  358.71  122.     26.    335.43  473.14
   39.43   14.71   27.86   14.43  582.     28.14   75.     29.14  156.
  193.43   19.29  187.71   66.    539.     46.43  516.    442.14  542.86
  520.14  442.86  376.    422.57  126.86   97.    514.86   73.86   96.
   61.57  446.29   51.86   30.43  441.86  486.14  456.14  331.71  450.
  250.71  434.43  195.86   40.71  123.    110.    455.43  464.29   27.86
  100.71  387.14   34.43   63.71  208.    106.14   90.71  442.     22.14
  104.43    8.71  406.71   17.86  172.43  108.43  380.57  413.     80.
  132.86  229.29   42.29  417.29  368.    281.14  142.71   29.71   51.14
  373.86   79.71  393.14  347.    374.43  380.     38.86  105.57  388.14
  327.14    7.71   14.71  112.43  492.86   34.29  186.     21.86   76.14
   29.86   31.    150.57   22.29   67.71   24.43  616.57    8.29   22.14
   32.71   13.29    9.     40.43   30.    559.57   47.71   

In [ ]:
#Input: X and Y matrices
#Output: weights vector calculated by linear regression
def linear_regression(matrixX,matrixY):
    XT=matrixX.T
    print(XT.shape)
    XTX=np.dot(XT,matrixX)
    XTY=np.dot(XT,matrixY)
    #inverse=np.linalg.inv(XTX)
    #weights=np.dot(inverse,XTY)

    #Avoid singular matrix
    c,resid,rand,sigma=np.linalg.lstsq(XTX,XTY)
    return c

#Input: X,Y,weights vector
#Output: the error of the weights vector
def error(matrixX,matrixY,weightsVector):
	Xw=np.dot(matrixX,weightsVector)
	YXw=matrixY-Xw
	YXwT=YXw.T
	result=np.dot(YXwT,YXw)
	return result

#K-fold validation


In [514]:
def separate(X,Y,K1,K2):
    xtrain=[]
    xvalid=[]
    ytrain=[]
    yvalid=[]
    for i in range(len(X)):
        if i not in range(K1,K2):
            xtrain.append(X[i])
            ytrain.append(Y[i])
        else:
            xvalid.append(X[i])
            yvalid.append(Y[i])
    return xtrain,ytrain,xvalid,yvalid
def LRcross_validation(X,Y,K):
	length=len(X)
	print(length)
	subLength=int(length/K)

	minD=None
	minWeights=None
	ErrorSum=0

	for i in range(K):
		k1=i*subLength
		k2=(i+1)*subLength
		trainingX,trainingY,testX,testY=separate(X,Y,k1,k2)
		trainX=np.array(trainingX)
		trainY=np.array(trainingY)
		LRweights=linear_regression(trainX,trainY)
		LRerror=error(testX,testY,LRweights)
		print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
		print (LRerror)
		print (LRweights)
		ErrorSum=ErrorSum+LRerror
		if minD is None:
			minD=LRerror
			minWeights=LRweights
		if LRerror<minD:
			minD=LRerror
			minWeights=LRweights

	NewError=ErrorSum/K
	
	return minWeights,NewError



In [515]:
# # #Test linear regression with normalization
# # LR=linear_regression(X,Y)
# # LRError=error(X,Y,LR)

# #np.savetxt("fooo.csv", LR, delimiter=",")
# print "The weight vector by linear regression: ", LR
# print "The error for the weight vector: ", LRError

# #Test lineat regression with K-fold cross-validation
# #Not needed


X=np.array(X,dtype=float)

Y=np.array(ytrain)
colomn=Y[:,1]
y=[0 if x=='NA' else x for x in colomn]
y=np.array([y],dtype=float)
y=y.T
val=0
for i in range(len(X)):
    if 'COMPLETE_REMISSION' in Y[i]:
        val+=1
        
xx=np.zeros((val,265))
yy=np.zeros((val,1))
iter=0
for i in range(len(X)):
    if 'COMPLETE_REMISSION' in Y[i]:
        xx[iter]=X[i]
        yy[iter]=y[i]
        iter+=1
print(len(xx))
LRcvw,LRcve=LRcross_validation(xx,yy,5)

115
115
(265, 92)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
[[ 3565682.01182298]]
[[  1.18920966e+02]
 [ -6.87280254e-01]
 [  1.19530178e+00]
 [ -1.35980932e+00]
 [  9.44185423e+00]
 [ -3.02288686e+01]
 [ -3.82723245e+00]
 [ -1.14249374e+01]
 [  1.24894622e+01]
 [  1.84956524e+00]
 [  2.50060213e+00]
 [ -1.09268096e+00]
 [  2.13950722e-04]
 [ -3.38306492e+00]
 [  2.27524843e+00]
 [  2.22186546e+01]
 [  4.84925642e-01]
 [  1.54033424e+00]
 [ -6.24737439e+00]
 [ -3.29649401e+00]
 [  3.01130663e-01]
 [ -1.16509275e-03]
 [  4.59715789e+00]
 [  2.12469879e-01]
 [  7.09038075e+00]
 [ -7.28674659e-02]
 [ -8.25852122e-01]
 [  1.00822211e+00]
 [  9.23471234e-01]
 [  1.05015489e+00]
 [  6.27902721e+00]
 [  8.72556136e+00]
 [  3.99382959e-01]
 [ -4.41034676e-01]
 [  1.09765435e+01]
 [  3.60813994e+00]
 [  8.34001731e+00]
 [ -4.70073930e+00]
 [  2.35739051e-01]
 [ -1.45030527e+01]
 [ -3.12970987e+00]
 [ -4.41497839e+00]
 [  9.96823281e+00]
 [ -1.68635981e+01]
 [  1.54525082e+01]
 [  1.04966779e+01]
 [ -2.

In [524]:
x = [i[0] for i in LRcvw]
print(x)

[-112.05198879587658, -0.9661049179293989, 0.56728612183646088, -22.708811860016993, -22.482194980247314, -17.850491194420865, -3.6759593786131655, -2.6012766331523025, 4.5249422395946794, -26.291321858437492, 3.7837939069573419, -4.926363536943537, 0.0042268681883976811, -2.6487434434475059, 7.9460907451369085, 11.701167156465178, 1.0119963606888298, -1.4591997617686836, -4.254770647508284, 30.478955435392635, -0.60636674159034243, -0.016415175918215397, 37.564618091837389, -7.0790889340686967, 19.634321772971678, -0.032097808301694286, -1.03309261950033, 0.61352260783110069, 0.29533988371578523, -0.50758060192107024, -1.4502657941026698, 25.842917034492494, 1.2984793714552669, -0.10952510248322667, 10.071002775949591, -0.89680926475075484, 20.640476151232463, 13.855480566053275, 4.625687106466394, -3.9005429723115945, -16.042932739650404, -6.7850696165303619, 2.0949849931132163, -14.129820374454834, 14.773096962675904, 22.108849792083305, 8.0230156140027304, 11.699014789836452, -16.1

In [525]:
X=np.array(patient,dtype=float)
Y=np.array(ytrain)
y1=Y[:,2]
y1=np.array(y1,dtype=float)
print(y1)

LRcvw,LRcve=LRcross_validation(X,y1,5)

[ 568.57  185.86   56.29   98.14  454.71   50.    734.86  690.14  410.14
  703.86   75.43   26.71  205.43  358.71  122.     26.    335.43  473.14
   39.43   14.71   27.86   14.43  582.     28.14   75.     29.14  156.
  193.43   19.29  187.71   66.    539.     46.43  516.    442.14  542.86
  520.14  442.86  376.    422.57  126.86   97.    514.86   73.86   96.
   61.57  446.29   51.86   30.43  441.86  486.14  456.14  331.71  450.
  250.71  434.43  195.86   40.71  123.    110.    455.43  464.29   27.86
  100.71  387.14   34.43   63.71  208.    106.14   90.71  442.     22.14
  104.43    8.71  406.71   17.86  172.43  108.43  380.57  413.     80.
  132.86  229.29   42.29  417.29  368.    281.14  142.71   29.71   51.14
  373.86   79.71  393.14  347.    374.43  380.     38.86  105.57  388.14
  327.14    7.71   14.71  112.43  492.86   34.29  186.     21.86   76.14
   29.86   31.    150.57   22.29   67.71   24.43  616.57    8.29   22.14
   32.71   13.29    9.     40.43   30.    559.57   47.71   

In [527]:
print(LRcvw.tolist())

[-360.7518845166698, -0.2531156592459481, -2.7527342477111274, -11.979841115090451, -35.476258934815284, 40.94357530274562, 23.159319183008233, -9.501565717892282, -30.49107472722164, -52.60896062406445, 31.767641349838293, -2.8439531089901413, 0.0022911952586355255, 0.7538338974062828, 5.105790670469085, 15.522263924923061, 0.6610756913600664, 0.9283271622225442, -0.22710841279835847, 18.23217601645662, -0.41367549119743785, -0.004213491896742028, 87.20514075776582, 45.53795270815176, 97.0569965732574, -0.24284851063403323, 1.2048104153829655, -0.8149950286181848, -0.5614483285587539, -1.1838859078662125, 6.21546366383992, 6.833327093328329, 1.4619096516508066, 0.2923631305539729, -14.865301387152643, -6.953237299610201, -11.625527381515244, 34.45766188458484, 23.504432080811682, -45.1156074835994, -24.091677720139966, 44.023520028427924, -6.9446366915406115, -43.65933700340407, 3.915823769174601, 29.493895211233532, 22.07857418842095, 13.157181373358899, 3.8120458364530085, 68.724560